In [1]:
from pathlib import Path
from kedro.framework.session import KedroSession
from kedro.framework.session.session import _activate_session
from kedro.framework.startup import _add_src_to_path

current_dir = Path.cwd()  # this points to 'notebooks/' folder
project_path = current_dir.parent  # point back to the root of the project
_add_src_to_path(project_path/"src", project_path)

session = KedroSession.create("csmr_kedro", project_path)
_activate_session(session)
context = session.load_context()
catalog = context.catalog


/home/matheus/.local/lib/python3.8/site-packages/pkg_resources/__init__.py:1130: DeprecationWarning: Use of .. or absolute path in a resource path is not allowed and will raise exceptions in a future release.
  return get_provider(package_or_requirement).get_resource_filename(


2021-12-27 23:11:47,047 - root - INFO - Registered hooks from 2 installed plugin(s): kedro-mlflow-0.7.2
2021-12-27 23:11:47,048 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.


In [3]:
catalog.load("train_features")

2021-12-27 23:12:03,320 - kedro.io.data_catalog - INFO - Loading data from `train_features` (CSVDataSet)...


,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f762,f763,f764,f765,f766,f767,f768,text,company,target
0,0.218544,-0.059096,-0.087296,0.101487,-0.080620,-0.001090,0.610474,-0.227217,-0.020762,-0.087030,...,0.492525,0.523683,0.133101,0.140130,-0.206967,0.260036,-0.025468,Riot é da China. As pessoas que estão acusando...,Riot Games,0.0
1,0.125816,-0.247009,-0.036374,0.425047,0.260109,-0.269446,0.858746,0.403927,0.332358,-0.286561,...,-0.106895,-0.107753,-0.070262,-0.106091,0.139984,-0.152723,-0.257614,perdao como fui vacilona ctg mas ó Aproveite m...,Magazine Luiza,0.5
2,0.188318,-0.112590,-0.076310,0.240603,0.227864,-0.378432,0.963279,0.252382,0.265371,-0.145545,...,0.081702,0.137503,-0.180536,-0.360719,0.003917,-0.285313,-0.005518,O MELIUZ TÁ DANDO 5 REAIS PARA QUE SE CADRASTR...,Magazine Luiza,0.5
3,-0.112302,-0.103360,0.128516,0.072404,0.026124,0.117978,0.314252,-0.258593,0.063542,-0.026962,...,0.401401,-0.239889,0.482421,0.084575,0.093814,0.027781,-0.055864,@marian4emanuele amando a marvel,Marvel,1.0
4,0.197553,0.070640,-0.115926,0.258791,0.090828,-0.347015,0.406229,0.315087,0.439300,-0.212995,...,0.280974,0.029056,-0.032374,-0.316548,0.152706,-0.393567,0.086916,"Box ""Magisterium"" com brinde e marcadores de p...",Magazine Luiza,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,-0.086634,-0.135602,0.125128,0.107106,-0.104425,0.008446,-0.063315,-0.371403,0.089333,-0.022558,...,0.645596,-0.274639,0.395577,0.134011,0.214625,0.077663,0.054454,O que é economia comportamental? Ela explica s...,Nubank,0.5
636,0.005592,-0.096058,0.143292,-0.008084,-0.071264,0.065238,0.226961,-0.274313,0.055363,-0.056693,...,0.397274,-0.402100,0.411537,0.122320,0.208670,0.081014,-0.083939,Essa bolsa caindo vou pagar a fatura da nubank,Nubank,0.0
637,0.142003,-0.129970,-0.014842,0.098226,-0.191507,-0.049907,0.782849,0.120403,0.284153,-0.084903,...,-0.187700,0.126388,0.131204,-0.018337,-0.379527,0.000851,-0.051973,Na moral quem não gosta de X-men pra mim autom...,Marvel,1.0
638,-0.056520,-0.078171,0.170138,-0.013296,-0.128705,0.120502,0.217074,-0.253330,0.023850,-0.000009,...,0.677436,-0.388043,0.465460,0.120801,0.115361,0.045337,0.013597,minha filosofia de vida agora é essa.,Nubank,0.5


In [2]:
catalog.load('torch_model')

2021-12-27 20:37:43,470 - kedro.io.data_catalog - INFO - Loading data from `torch_model` (TorchDataSet)...


Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertSA(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29794, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
     